# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

# Observations
1. From the humidity heatmap we can see that cities close to the ocean tend to have higher humidity level, inner land cities have lower humidity.
2. Peru, South Africa, Japan, New Zealand, Spain are countries with higher humidity level compare to other countries. 
3. Countires in northern Africa and Asia showed very low humidity level. 

In [91]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import string

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.DataFrame(pd.read_csv('output_data/cities.csv'))
cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Angoulême,45.6500,0.1500,54.21,91,100,3.44,FR,1651211301
1,1,Fethiye,36.6217,29.1164,62.47,46,1,4.12,TR,1651211302
2,2,Mocuba,-16.8392,36.9856,67.93,90,62,5.08,MZ,1651211302
3,3,Acatlán,20.1500,-98.4500,57.72,76,61,2.33,MX,1651211304
4,4,La Ronge,55.1001,-105.2842,41.25,87,100,6.91,CA,1651211126


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [106]:
gmaps.configure(api_key=g_key)
humidity = cities_df['Humidity']
humidity.max()

99

In [111]:
locations = cities_df[['Lat', 'Lng']]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating= False, max_intensity= 300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [95]:
# Drop any rows with nul values
clean_cities_df = cities_df.dropna(how = 'any')

# Filter the cities using the following conditions:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

filtered_cities_df = clean_cities_df.loc[(clean_cities_df['Max Temp'] > 70) & (clean_cities_df['Max Temp'] < 80) & (clean_cities_df['Wind Speed'] < 10) & (clean_cities_df['Cloudiness'] == 0)]
filtered_cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,East London,-33.0153,27.9116,74.34,46,0,8.05,ZA,1651211305
44,44,Saint George,37.1041,-113.5841,70.39,8,0,6.91,US,1651211126
134,136,Coahuayana Viejo,18.7333,-103.6833,72.18,79,0,1.57,MX,1651211389
242,245,Erzin,36.9559,36.2000,74.64,47,0,1.68,TR,1651211462
267,271,Belyy Yar,53.6039,91.3903,71.49,20,0,8.95,RU,1651211479
414,421,Taoudenni,22.6783,-3.9836,77.02,13,0,7.61,ML,1651211575
552,562,Buckeye,33.3703,-112.5838,78.03,24,0,8.68,US,1651211660


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [102]:
# Store the city name, latitude, longitude into a data frame
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng']].copy()

# Add a new column "Hotel Name" to the data frame
hotel_df['Hotel Name'] = ""

hotel_df.reset_index(drop=True)

,City,Country,Lat,Lng,Hotel Name
0,East London,ZA,-33.0153,27.9116,
1,Saint George,US,37.1041,-113.5841,
2,Coahuayana Viejo,MX,18.7333,-103.6833,
3,Erzin,TR,36.9559,36.2000,
4,Belyy Yar,RU,53.6039,91.3903,
5,Taoudenni,ML,22.6783,-3.9836,
6,Buckeye,US,33.3703,-112.5838,


In [103]:
# Create a params dict that will update with new city each iteration
params = {
    "key" : g_key,
    "radius" : 5000,
    "types": "lodging",
    }

# Loop through the hotel_df and run a lat/lng search for each city
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']

    params['location'] = f'{lat},{lng}'

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    hotel_name = requests.get(base_url, params=params)

    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel_name["results"][0]["name"]
    
    except (KeyError, IndexError): 
        print("Missing field/result... skipping")


Missing field/result... skipping


In [104]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [105]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info )
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))